In [ ]:
%pip install google-genai

In [ ]:
#configura a API KEY do Google Gemini
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
#utilizando o Google Search para fazer pesquisa no google

response = client.models.generate_content(
    model=MODEL_ID,
    contents="qual dia é hoje?",
    config={"tools": [{"google_search": {}}]}
)

print(response.text)

Hoje é sexta-feira, 16 de maio de 2025. É o 136º dia do ano.


In [ ]:
# Instalar Framework ADK de agentes do Google

!pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [72]:
from IPython.display import display, Markdown

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [66]:
def agente_identificador_de_residuo(entrada):
    identificador = Agent(
        name="agente_identificador_residuo",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente especialista em identificar tipos de resíduos em descrições ou perguntas de usuários.
        Use a ferramenta de pesquisa do Google para entender se existe um tipo de resíduo associado ao conteúdo fornecido.
        Retorne apenas o tipo de resíduo, como por exemplo: "pilhas", "lixo eletrônico", "vidro", "papel", "óleo de cozinha", etc.
        Se não conseguir identificar claramente, responda: "false".
        """,
        description="Agente que identifica o tipo de resíduo a partir de uma pergunta ou descrição.",
        tools=[google_search]
    )

    entrada_do_agente = f"Texto do usuário: {entrada}"
    resposta = call_agent(identificador, entrada_do_agente)

    if resposta and resposta.strip().lower() != "false":
        return resposta.strip()
    else:
        return "false"


In [54]:
# --- Agente 2: Identificador da localização ---
def agente_identificador_de_localizacao(entrada):
    identificador = Agent(
        name="agente_identificador_localizacao",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente especialista em identificar localização ou endereços em descrições ou perguntas de usuários.
        Use a ferramenta de pesquisa do Google para entender se existe alguma localização real associada ao conteúdo fornecido.
        Retorne apenas um endereço ou localização, como por exemplo: "Rua Fernão Mendes Pinto, São Paulo 03802010", "Avenida Itaquera", etc.
        Se não conseguir identificar claramente, responda: "false"
        """,
        description="Agente que identifica a localização a partir de uma pergunta ou descrição.",
        tools=[google_search]
    )

    entrada_do_agente = f"Texto do usuário: {entrada}"
    resposta = call_agent(identificador, entrada_do_agente)

    if resposta and resposta.strip().lower() != "false":
        return resposta.strip()
    else:
        return "false"


In [70]:
# --- Agente 3: Identificador de ecopontos próximos à localização que permitem o descarte do resíduo identificado ---
def agente_identificador_de_ecopontos(localizacao, residuo):
    identificador = Agent(
        name="agente_identificador_ecopontos",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente especialista em identificar se existem ecopontos próximos à localização informada que permitam o descarte do resíduo fornecido.
        Use a ferramenta de pesquisa do Google para verificar se há algum ecoponto adequado próximo à localização fornecida.
        Retorne apenas os 3 ecopontos mais próximos se houver, caso contrário retorne a melhor opção, formatados da seguinte forma:
        "Nome do ecoponto, endereço completo".
        Caso não consiga identificar claramente, responda: "false"
        """,
        description="Agente que identifica ecopontos próximos à localização para descarte do resíduo.",
        tools=[google_search]
    )

    entrada_do_agente = f"Resíduo: {residuo}, Localização: {localizacao}"
    resposta = call_agent(identificador, entrada_do_agente)

    if resposta and resposta.strip().lower() != "false":
        return resposta.strip()
    else:
        return "false"


In [92]:
from IPython.display import display, Markdown

# --- Fluxo completo ---
def fluido_do_app():
    # Boas-vindas
    display(Markdown("### 👋 Bem-vindo ao app de descarte inteligente de resíduos!"))
    print("Qual o resíduo a ser descartado?")
    residuo_input = input("Resíduo: ")
    residuo = agente_identificador_de_residuo(residuo_input)

    while residuo == "false":
        print("\n⚠️ Oops! Não consegui identificar esse resíduo.")
        print("Tente algo como: 'pilha', 'óleo de cozinha', 'celular'...")
        residuo_input = input("Resíduo: ")
        residuo = agente_identificador_de_residuo(residuo_input)

    print(f"\n✅ Resíduo identificado: {residuo}")

    # Etapa 2: Identificar localização
    print("\nQual a localização aproximada onde você prefere fazer o descarte?")
    print("Pode ser uma rua, avenida ou bairro.")
    endereco_input = input("Endereço: ")
    endereco = agente_identificador_de_localizacao(endereco_input)

    while endereco == "false":
        print("\n⚠️ Oops! Não consegui identificar essa localização.")
        print("Tente fornecer um endereço mais claro ou outro local próximo.")
        endereco_input = input("Endereço: ")
        endereco = agente_identificador_de_localizacao(endereco_input)

    print(f"\n✅ Localização identificada: {endereco}")
    print("\n🔍 Buscando ecopontos próximos...")

    # Etapa 3: Buscar ecopontos
    ecopontos = agente_identificador_de_ecopontos(endereco, residuo)

    while ecopontos == "false":
        print("\n⚠️ Não consegui encontrar ecopontos próximos.")
        print("Tente fornecer outro endereço ou região.")
        endereco_input = input("Novo endereço: ")
        endereco = agente_identificador_de_localizacao(endereco_input)
        if endereco != "false":
            ecopontos = agente_identificador_de_ecopontos(endereco, residuo)

    # Resultado final
    print("\n✅ Ecopontos próximos encontrados:")
    print(ecopontos)

    print("\n✅ Obrigado por usar o app de descarte inteligente de resíduos!")
    print("Tenha um ótimo dia!")


In [94]:
ecopontos = fluido_do_app()

### 👋 Bem-vindo ao app de descarte inteligente de resíduos!

Qual o resíduo a ser descartado?
Resíduo: celular

✅ Resíduo identificado: Lixo eletrônico.

Qual a localização aproximada onde você prefere fazer o descarte?
Pode ser uma rua, avenida ou bairro.
Endereço: Avenida Boturussu

✅ Localização identificada: Avenida Boturussu, São Paulo

🔍 Buscando ecopontos próximos...

✅ Ecopontos próximos encontrados:
Com base nas informações encontradas, aqui estão algumas opções para descarte de lixo eletrônico próximo à Avenida Boturussu, São Paulo:

1.  **Ecoponto Boturussu:** Rua Nélio Batista Guimarães, 183. No entanto, vale ressaltar que os ecopontos não recebem lixo eletrônico.

Considerando que os Ecopontos não aceitam lixo eletrônico, outras opções incluem:

*   **Sucata Digital:** Oferece coleta gratuita de resíduos eletrônicos em empresas e residências na Grande São Paulo. A Matriz da Sucata Digital está localizada na Rua Templários, 192, Vila Formosa.
*   **Green Eletron:** Possui diversos pontos de coleta na cidade de São Paulo. Você pode en